In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import os
from moviepy.editor import VideoFileClip
from IPython import display

%matplotlib inline

In [ ]:
def grayscale(img):
    return cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
def canny(img, low_threshold, high_threshold):
    return cv.Canny(img, low_threshold, high_threshold)

In [ ]:
def gaussian_blur(img, kernel_size):
    return cv.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [ ]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv.bitwise_and(img, mask)
    
    return masked_image

In [ ]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    draw_right = True
    draw_left = True
    
    slope_threshold = 0.5
    slopes = []
    new_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        
        if x2 - x1 == 0.: 
            slope = 999.
        else:
            slope = (y2 - y1) / (x2 - x1)
            
        if abs(slope) > slope_threshold:
            slopes.append(slope)
            new_lines.append(line)
        
    lines = new_lines
    
    right_lines = []
    left_lines = []
    for i, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]
        img_x_center = img.shape[1] / 2
        if slopes[i] > 0 and x1 > img_x_center and x2 > img_x_center:
            right_lines.append(line)
        elif slopes[i] < 0 and x1 < img_x_center and x2 < img_x_center:
            left_lines.append(line)
            
    right_lines_x = []
    right_lines_y = []
    
    for line in right_lines:
        x1, y1, x2, y2 = line[0]
        
        right_lines_x.append(x1)
        right_lines_x.append(x2)
        
        right_lines_y.append(y1)
        right_lines_y.append(y2)
        
    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y, 1)
    else:
        right_m, right_b = 1, 1
        draw_right = False
        
    left_lines_x = []
    left_lines_y = []
    
    for line in left_lines:
        x1, y1, x2, y2 = line[0]
        
        left_lines_x.append(x1)
        left_lines_x.append(x2)
        
        left_lines_y.append(y1)
        left_lines_y.append(y2)
        
    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y, 1)
    else:
        left_m, left_b = 1, 1
        draw_left = False
    
    y1 = img.shape[0]
    y2 = img.shape[0] * (1 - trap_height)
    
    right_x1 = (y1 - right_b) / right_m
    right_x2 = (y2 - right_b) / right_m
    
    left_x1 = (y1 - left_b) / left_m
    left_x2 = (y2 - left_b) / left_m
    
    y1 = int(y1)
    y2 = int(y2)
    right_x1 = int(right_x1)
    right_x2 = int(right_x2)
    left_x1 = int(left_x1)
    left_x2 = int(left_x2)
    
    if draw_right:
        cv.line(img, (right_x1, y1), (right_x2, y2), color, thickness)
    if draw_left:
        cv.line(img, (left_x1, y1), (left_x2, y2), color, thickness)

In [ ]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
   
    return line_img

In [ ]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv.addWeighted(initial_img, α, img, β, λ)

In [ ]:
def filter_colors(image):
    white_threshold = 200
    lower_white = np.array([white_threshold, white_threshold, white_threshold])
    upper_white = np.array([255, 255, 255])
    white_mask = cv.inRange(image, lower_white, upper_white)
    white_image = cv.bitwise_and(image, image, mask=white_mask)

    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    lower_yellow = np.array([90,100,100])
    upper_yellow = np.array([110,255,255])
    yellow_mask = cv.inRange(hsv, lower_yellow, upper_yellow)
    yellow_image = cv.bitwise_and(image, image, mask=yellow_mask)

    image2 = cv.addWeighted(white_image, 1., yellow_image, 1., 0.)

    return image2

In [ ]:
kernel_size = 3

low_threshold = 50
high_threshold = 150

trap_bottom_width = 0.85
trap_top_width = 0.07
trap_height = 0.4

rho = 2
theta = 1 * np.pi/180
threshold = 15
min_line_length = 10
max_line_gap = 20

In [ ]:
def annotate_image(image_in):
    image = filter_colors(image_in)
    
    gray = grayscale(image)

    blur_gray = gaussian_blur(gray, kernel_size)

    edges = canny(blur_gray, low_threshold, high_threshold)

    imshape = image.shape
    vertices = np.array([[\
        ((imshape[1] * (1 - trap_bottom_width)) // 2, imshape[0]),\
        ((imshape[1] * (1 - trap_top_width)) // 2, imshape[0] - imshape[0] * trap_height),\
        (imshape[1] - (imshape[1] * (1 - trap_top_width)) // 2, imshape[0] - imshape[0] * trap_height),\
        (imshape[1] - (imshape[1] * (1 - trap_bottom_width)) // 2, imshape[0])]]\
        , dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    initial_image = image_in.astype('uint8')
    annotated_image = weighted_img(line_image, initial_image)
    
    return annotated_image

In [ ]:
test_images = os.listdir('test_images/')

for image in test_images:
    annotated_image = annotate_image(mpimg.imread((os.path.join('test_images', image))))
    fname = image.split('.')[0] + '_annotated.jpg'
    plt.imsave(os.path.join('output_images', fname), annotated_image)

In [ ]:
output_images = os.listdir('output_images/')
for image in output_images:
    fname = ('output_images/'+image)
    print('\n\n************* '+image+' *************\n\n')
    img = mpimg.imread(fname)
    imgplot = plt.imshow(img)
    plt.show()


In [ ]:
def process_image(image):
    result = annotate_image(image)

    return result

In [ ]:
test_videos = os.listdir('test_videos/')

for video in test_videos:
    fname = 'output_videos/' + video.split('.')[0] + '_output.mp4'
    clip = VideoFileClip('test_videos/'+video)
    output_video = clip.fl_image(process_image)
    %time output_video.write_videofile(fname, audio=False)

In [ ]:
output_videos = os.listdir('output_videos/')
for video in output_videos:
    videoVid = ('output_videos/'+video)
    print('\n\n******************************************** '+video+' ********************************************\n\n')
    display.display(display.Video(videoVid, embed=True))